# find out what are the events in which detector='unknown' from $^{252}$Cf fission
$^{252}$Cf source activity is 1 uC

In [1]:
import sys; 
sys.path.insert(0, '/Users/erezcohen/Desktop/PETsys/Software/PETsysAnalysis/Python'); 
from PETsys_analysis_tools import *;
sys.path.insert(0, '/Users/erezcohen/Desktop/PETsys/Software/PETsysAnalysis/Python/BoxSi'); 
from proto21_tools import *;
%config InlineBackend.figure_format = 'retina'

In [16]:
dT = 10; # sec
Cf252_groups = pd.read_csv('/Users/erezcohen/Desktop/data/PETsys/BoxSi_proto2.1/Cf252_%dsec_group.dat'%dT,
                            delimiter='\t',
                            names=['N(SiPMs)','n(SiPM)','time','charge','channel'])
Cf252_groups

,N(SiPMs),n(SiPM),time,charge,channel
0,8,0,888325107579,6.372524,1000
1,8,1,888325108711,6.135490,1018
2,8,2,888325108628,5.290455,1022
3,8,3,888325108110,4.761852,1006
4,8,4,888325108222,4.538231,1004
5,8,5,888325107539,3.651077,1001
6,8,6,888325108658,3.612843,1012
7,8,7,888325108653,2.653442,1011
8,1,0,897666556628,4.174099,188
9,2,0,899394479487,2.556427,802


## group cosmic event-groups into single "events" detected in each detector

In [51]:
def process_SiPMgroups_to_events(groups, NSiPM_min=3, dT=100, fdebug=0):#{
    '''
    input:
    --------
    groups      pandas.DataFrame
    NSiPM_min   minimal number of allowed SiPMs that fired per event
    
    return:
    --------
    events  pandas.DataFrame
    '''    
    import sys;  sys.path.insert(0, '/Users/erezcohen/Desktop/PETsys/Software/PETsysAnalysis/Python'); 
    from PETsys_analysis_tools import *;

    dataset = groups

    event_idx = 0
    N = dataset['N(SiPMs)'][0]
    Qtot, t, channels, detector = 0, [], [], 0
    events = pd.DataFrame() #events = dict()
    
    for i,SiPMdata in dataset.iterrows():#{
        
        # iterate over all SiPMs that fired in this event
        N,n,ch = SiPMdata['N(SiPMs)'], SiPMdata['n(SiPM)'], SiPMdata['channel']
        Q = SiPMdata['charge']; 
        if Q>0: Qtot = Qtot + lineariseChargeDeposited(Q); 
        else: Qtot = Qtot + Q
        t.append(SiPMdata['time'])
        channels.append(SiPMdata['channel'])

        if fdebug>2: print N,n,ch,Q

        # stop iterating if we reached a new event
        if ((n == N-1) or (i==len(dataset)-1)):#{ 
            # determine which detector was hit
            # print (np.min(channels),np.max(channels))
            if (64 <= np.min(channels)) and (np.max(channels)<=127):
                detector = 12;
                detector_type = 'KETEK 3x3 - 1';
            elif (128 <= np.min(channels)) and (np.max(channels)<=192):
                detector = 3;
                detector_type = 'KETEK 3x3 - 2';
            elif (895 <= np.min(channels)) and (np.max(channels)<=1023):
                detector = 6;
                detector_type = 'SensL 6x6 - 1';
            elif (767 <= np.min(channels)) and (np.max(channels)<=895):
                detector = 9;
                detector_type = 'SensL 6x6 - 2';
            else:
                detector = 0; # unknown
                detector_type = 'unknown';

            event_df = pd.DataFrame(data={'event':[event_idx],'N(SiPMs)':[N],
                                              'detector':[detector],'det.type':[detector_type],
                                              'channels':[[channels]],                        
                                              'time[ns]':[np.min(t)/ns],'Qtot':[Qtot]})
            if fdebug>1: #{
                print('------------------------------')
                print(event_df[['event','detector','N(SiPMs)','channels','time[ns]','Qtot']]);                 
            #}
                
            # if event is valid, plug into a Pandas.DataFrame
            valid_event = False
            if (N>=NSiPM_min) and (Qtot>0): #{
                valid_event = True
            #}
            if valid_event:#{
                if fdebug>1: print('valid event')
                events = events.append([event_df], ignore_index=True)
                event_idx = event_idx+1
            #}
            # initialize next event-group            
            Qtot, t, channels, detector = 0, [], [], 0        
            if fdebug>1: print('------------------------------')
        #}        
        if (i%(len(dataset)/10)==0): print '%.1f'%(100*float(i)/len(dataset)),'%'
    #}
    
    KETEK_events = events[(events['detector']==3) | (events['detector']==12)]
    SensL_events = events[(events['detector']==6) | (events['detector']==9)]
    print(len(events),'events during',dT,'seconds, with average rate %.2f+/-%.2f Hz'%(len(events)/dT,sqrt(len(events))/dT))
    print('%.1f'%(100.*len(KETEK_events[KETEK_events.detector==12])/len(events))+'% in detector 12 (KETEK)')
    print('%.1f'%(100.*len(KETEK_events[KETEK_events.detector==3])/len(events))+'% in detector 3 (KETEK)')
    print('%.1f'%(100.*len(SensL_events[SensL_events.detector==6])/len(events))+'% in detector 6 (SensL)')
    print('%.1f'%(100.*len(SensL_events[SensL_events.detector==9])/len(events))+'% in detector 9 (SensL)')
    print('%.1f'%(100.*len(events[events.detector==0])/len(events))+'% in unknown detector')

    
    return events,KETEK_events,SensL_events
#}

<ipython-input-51-88e9ee0fd077>:1: SyntaxWarning: import * only allowed at module level
  def process_SiPMgroups_to_events(groups, NSiPM_min=3, dT=100, fdebug=0):#{


In [53]:
events,KETEK_events,SensL_events = process_SiPMgroups_to_events(Cf252_groups, dT=dT, 
                                                                NSiPM_min=2,fdebug=0);
events[0:10]

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %
(1886, 'events during', 10, 'seconds, with average rate 188.00+/-4.34 Hz')
5.0% in detector 12 (KETEK)
4.7% in detector 3 (KETEK)
48.7% in detector 6 (SensL)
41.4% in detector 9 (SensL)
0.2% in unknown detector


,N(SiPMs),Qtot,channels,det.type,detector,event,time[ns]
0,8.0,27.756380,"[[1000.0, 1018.0, 1022.0, 1006.0, 1004.0, 1001...",SensL 6x6 - 1,6,0,8.883251e+08
1,2.0,6.877845,"[[898.0, 909.0]]",SensL 6x6 - 1,6,1,9.027549e+08
2,52.0,224.103830,"[[966.0, 964.0, 957.0, 937.0, 951.0, 940.0, 93...",SensL 6x6 - 1,6,2,9.035975e+08
3,53.0,101.430670,"[[135.0, 134.0, 186.0, 140.0, 132.0, 152.0, 18...",KETEK 3x3 - 2,3,3,9.035975e+08
4,2.0,5.052344,"[[68.0, 70.0]]",KETEK 3x3 - 1,12,4,9.085693e+08
5,2.0,3.480885,"[[907.0, 923.0]]",SensL 6x6 - 1,6,5,9.122172e+08
6,2.0,7.392913,"[[945.0, 944.0]]",SensL 6x6 - 1,6,6,9.139953e+08
7,10.0,31.055885,"[[937.0, 939.0, 940.0, 941.0, 936.0, 942.0, 94...",SensL 6x6 - 1,6,7,9.229411e+08
8,2.0,4.080690,"[[821.0, 822.0]]",SensL 6x6 - 2,9,8,9.255368e+08
9,2.0,8.867074,"[[99.0, 100.0]]",KETEK 3x3 - 1,12,9,9.286339e+08


In [56]:
unknown_events = events[events.detector==0]
unknown_events[['event','channels']]

,event,channels
164,164,"[[1018.0, 987.0, 898.0, 964.0, 958.0, 805.0]]"
1195,1195,"[[810.0, 811.0, 993.0, 1000.0, 997.0, 999.0, 9..."
1635,1635,"[[803.0, 1006.0, 1004.0, 802.0, 807.0, 810.0, ..."


In [62]:
unknown_events['channels'][164]

[[1018.0, 987.0, 898.0, 964.0, 958.0, 805.0]]

In [64]:
unknown_events['channels'][1195]

[[810.0, 811.0, 993.0, 1000.0, 997.0, 999.0, 994.0, 1001.0]]

In [63]:
unknown_events['channels'][1635]

[[803.0,
  1006.0,
  1004.0,
  802.0,
  807.0,
  810.0,
  808.0,
  811.0,
  804.0,
  809.0,
  805.0,
  812.0,
  814.0,
  827.0,
  799.0,
  800.0,
  801.0,
  797.0,
  819.0,
  829.0,
  777.0,
  820.0]]